In [1]:
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity

from scipy.sparse import csr_matrix

import pickle

In [2]:
metadata = pd.read_csv('movies_metadata.csv')

metadata.head(3)

<ipython-input-2-0f47d7d862c8>:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata = pd.read_csv('movies_metadata.csv')


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


In [3]:
tfidf = TfidfVectorizer(max_features=10000,stop_words='english')


metadata['overview'] = metadata['overview'].fillna('')

tfidf_matrix = tfidf.fit_transform(metadata['overview'].values.astype('U'))

tfidf_sparse = csr_matrix(tfidf_matrix)

In [4]:
similarity_sparse = cosine_similarity(tfidf_sparse, dense_output=False)


In [5]:
df_indices = pd.Series(metadata.index, index = metadata['title']).drop_duplicates()

In [6]:
def get_recommendations(title, cosine_sim=similarity_sparse):
    # Ensure the title exists in df_indices
    idx = df_indices.get(title, -1)
    if idx == -1:
        return "Movie title not found."

    # Get similarity scores for all movies
    sim_scores = list(enumerate(cosine_sim[idx].toarray().flatten()))

    # Sort by similarity score (descending)
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get top 10 most similar movies (excluding itself)
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return top 10 recommended movie titles
    return metadata.iloc[movie_indices][['title']]


In [7]:
print(get_recommendations('The Dark Knight Rises'))

                                                   title
15511                         Batman: Under the Red Hood
585                                               Batman
150                                       Batman Forever
1328                                      Batman Returns
12481                                    The Dark Knight
21194  Batman Unmasked: The Psychology of the Dark Kn...
19792            Batman: The Dark Knight Returns, Part 1
9230                  Batman Beyond: Return of the Joker
35983                                  Batman: Bad Blood
18035                                   Batman: Year One


In [ ]:
from scipy.sparse import save_npz

# Split into chunks (example: save in 4 parts)
num_splits = 4
rows_per_split = similarity_sparse.shape[0] // num_splits

for i in range(num_splits):
    start = i * rows_per_split
    end = (i + 1) * rows_per_split if i != num_splits - 1 else similarity_sparse.shape[0]

    chunk = similarity_sparse[start:end]
    save_npz(f"similarity_sparse_part_{i}.npz", chunk)